# Applied Data Analytics

## Wedge Project

<!-- Any general commentary you'd like to say about the project --> 

### Task 1

* Files for this task: 
`Task 1 - Wedge.ipynb`:
Python code that connects to Big Query, unzips csv wedge files and saves them to a new folder, then reformats the files (i.e. adds in headers to files that don't have them, removes quotes and problematic values, then creates columns based on , or ; as a delimiter). Once data is cleaned, it is uploaded as a new table in Big Query.

### Task 2

* Files for this task: 
`Task 2 - Wedge.ipynb`:
Python code that takes a sample of owners and their full purchasing history, excluding card no 3 from Big Query. Then creates dataframe and saves the data back as a text file.

`output_data.txt`: 
The output file created from the python code in the "Task 2 -Wedge.ipynb" file

	

### Task 3

* Files for this task: 

`Task 3 - Wedge.ipynb`: 
Python code that creates a local database of 3 tables. One that summarizes the sales by date and hour, a table that summarizes sales by owner by year and month and a table that summarizes sales by product description by year and month.

`wedge-reporting`: 
Database created from the python code in file "Task 3 - Wedge.ipynb". This database includes 3 tables: "product-sales", "owner-sales" and "date-hour"


## Query Comparison Results

Fill in the following table with the results from the 
queries contained in `gbq_assessment_query.sql`. You only
need to fill in relative difference on the rows where it applies. 
When calculating relative difference, use the formula 
` (your_results - john_results)/john_results)`. 



|  Query  |  Your Results  |  John's Results | Difference | Rel. Diff | 
|---|---|---|---|---|
| Total Rows  | 85760139 |  85760139 | 0 | 0 |
| January 2012 Rows  | 1070907  | 1070907  | 0 | 0 |
| October 2012 Rows  | 1042287  |  1042287 | 0 | 0 |
| Month with Fewest  | February  | February  | No  | NA  |
| Num Rows in Month with Fewest  |  6556770 | 6556770 | 0 | 0 |
| Month with Most  | May  | May | No  | NA  |
| Num Rows in Month with Most  | 7578372  | 7578372  | 0 | 0 |
| Null_TS  | 7123792  | 0  | 7123792  |  1 |
| Null_DT  | 0 | 0 | 0 | 0 |
| Null_Local  | 234843  |  234839 | 4 | .000017 |
| Null_CN  | 0 | 0 | 0 | 0 |
| Num 5 on High Volume Cards  | 14987  | 14987  | Yes | NA  |
|  Num Rows for Number 5 | 460630  | 460625  | 5 | .000011 |
| Num Rows for 18736  |   |   |   |   |
| Product with Most Rows  |   |   | Yes/No  | NA  |
| Num Rows for that Product  |   |   |   |   |
| Product with Fourth-Most Rows  |   |   | Yes/No  | NA  |
| Num Rows for that Product  |   |   |   |   |
| Num Single Record Products  |   |   |   |   |
| Year with Highest Portion of Owner Rows  |   |   | Yes/No  | NA |
| Fraction of Rows from Owners in that Year  |   |   |   |   |
| Year with Lowest Portion of Owner Rows  |   |   | Yes/No  | NA |
| Fraction of Rows from Owners in that Year  |   |   |   |   |

## Reflections

<!-- I'd love to get 100-200 words on your experience doing the Wedge Project --> 

#SQL CHECK!
-- Total Rows: 85760139

SELECT count(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*` 
--FROM `your_project.your_dataset.your_table_prefix_*` 


-- January/Oct 2012 Rows: 1070907/1042287
SELECT EXTRACT(Year from datetime) as Yr,
       EXTRACT(Month from datetime) as Mo,
       count(*)
FROM `umt-msba.wedge_transactions.transArchive_*` 
--FROM `your_project.your_dataset.your_table_prefix_*` 
GROUP BY Yr, Mo
HAVING Yr = 2012 
ORDER BY Yr, Mo
-- January = ?
-- October = ?


-- Rows by Month: 6556770/7578372

SELECT EXTRACT(Month from datetime) as Mo,
       count(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*` 
--FROM `your_project.your_dataset.your_table_prefix_*` 
WHERE EXTRACT(Year from datetime) <= 2016
GROUP BY Mo
ORDER BY cnt
-- January = ?
-- October = ?


-- Null Counts

SELECT COUNTIF(trans_subtype is NULL) as Null_TS,
       COUNTIF(datetime is NULL) as Null_DT,
       COUNTIF(local IS NULL) as Null_Local,
       COUNTIF(card_no IS NULL) as Null_CN
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*` 
-- Each value

| Null_TS  | 7123792  | 0  |
| Null_DT  | 0 | 0 |
| Null_Local  | 234843  |  234839 |
| Null_CN  | 0 | 0 |


-- High volume cards

SELECT card_no, 
       COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*` 
WHERE card_no != 3
GROUP BY card_no
ORDER BY cnt desc
LIMIT 10
-- What card is #5 on the list and how many rows does it have?

Card No: 14987
Rows: 460630

-- 18736 Rows

SELECT card_no, 
       COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*` 
WHERE card_no = 18736
GROUP BY card_no
-- How many rows do I have? 


-- Popular Products

SELECT LOWER(Description), 
       COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*` 
WHERE trans_type = "I" AND
      Description != "Discount"
GROUP BY LOWER(Description)
ORDER BY cnt desc
LIMIT 10
-- Which product is #1? How many rows does it have? 
-- Which product is #4? How many rows does it have? 


-- Single-record items

SELECT 
	COUNT(DISTINCT Description) as SingleRecordItems
FROM (
	SELECT LOWER(Description) as Description,
		   COUNT(*) as cnt
	FROM `umt-msba.wedge_transactions.transArchive_*`
	--FROM `your_project.your_dataset.your_table_prefix_*` 
  WHERE trans_type = "I" AND Description != "Discount"
	GROUP BY Description
	HAVING cnt = 1
	)

-- Owner Fractions by Year	

SELECT
  Y,
  OwnerRows,
  NonOwnerRows,
  ROUND(OwnerRows/(OwnerRows+NonOwnerRows),4) AS OwnerFrac
FROM (
  SELECT
    EXTRACT(Year
    FROM
      datetime) AS Y,
    COUNTIF(card_no != 3) AS OwnerRows,
    COUNTIF(card_no = 3) AS NonOwnerRows
  FROM
    `umt-msba.wedge_transactions.transArchive_*`
    --`your_project.your_dataset.your_table_prefix_*` 
  GROUP BY
    Y)
ORDER BY
  Y
  
-- Which year has the highest portion of owner rows? What year? What fraction?
-- Which year has the lowest portion of owner rows? What year? What fraction?
